In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# N/mm2
f_mk = 28
f_vk = 3.5
f_c90k = 2.5
E0mean = 12600
E005 = 10500
Gmean = 650

gammaM = 1.45
kmod = 0.9

In [3]:
M_d = 257.87465409375 #kNm
V_d = 135.101325 #kN

M_d = M_d * 10**6 
V_d = V_d * 10**3 

### Flessione
$$ \sigma_{m,d} = \frac{M_d}{W}\leq f_{m,d} = \frac{k_{mod} \cdot f_{m,k}}{\gamma_M} $$ 

sbandamento
$$ \sigma_{m,d} \leq k_{crit} \cdot f_{m,d} $$ 

In [4]:
# mm
b = 200 
h = 700
l = 7635

In [5]:
W = b * h**2 / 6 #sezioni rettangolari
print(W)
J = b*h**3 / 12
print(J)

sigma_md = M_d / W
print(sigma_md)

16333333.333333334
5716666666.666667
15.788244128188776


In [6]:
f_md = kmod * f_mk / gammaM
f_md

17.379310344827587

In [7]:
sigma_md < f_md

True

##### Sbandamento

In [8]:
# Vedi pagina 291 del libro
l_t = l # distanza tra due ritegni torsionali successivi (non ne abbiamo)
a1 = 1.13
a2 = 1.44
az = h/2

# caso generico per il RETTANGOLO e non semplificato per h/b > 4
B = E0mean * b**3 * h / 12 # rigidezza flessionale attorno asse z
T = Gmean * b**3 * h / (3*(1 + 0.6 * b/h)) # rigidezza torsionale 
l_eff = l_t / (a1 * (1 - a2 * az/l_t * np.sqrt(B/T)))


sigma_mcrit = np.pi/l_eff * b**2 / h * E005 * np.sqrt(Gmean/E0mean)
lambda_relm = np.sqrt(f_mk/sigma_mcrit)
if lambda_relm < 0.75:
    k_crit = 1
elif lambda_relm < 1.14:
    k_crit = 1.56 - .75*lambda_relm
else:
    k_crit = 1/(lambda_relm**2)

print('l_eff: ', l_eff)
print('l_eff/l: ', l_eff/l)
print('sigma_mcrit: ', sigma_mcrit)
print('lambda_relm: ', lambda_relm)
print('k_crit: ', k_crit)

l_eff:  8017.669252492032
l_eff/l:  1.0501203998024926
sigma_mcrit:  53.397926375660944
lambda_relm:  0.7241304657198179
k_crit:  1


In [9]:
sigma_md < k_crit * f_md

True

### Taglio

In [10]:
#kcrit legno lamellare = 2.5/f_vk -> b_eff = kcr * b
b_eff = 2.5/f_vk * b
print(b_eff)

tau_d = V_d * 1.5 / (b_eff * h)
print(tau_d)

f_vd = kmod * f_vk / gammaM
print(f_vd)

142.85714285714286
2.026519875
2.1724137931034484


In [11]:
tau_d < f_vd

True

### Compressione appggio 90 sulla scarpa

In [12]:
f_c90d = f_c90k * kmod / gammaM

In [21]:
l_appoggio = 80
A_ef = b* (l_appoggio+30+0) # min(30mm , a , l_appoggio, l/2) sx e dx
k_c90 = 1.75

sigma_c90_appoggio = V_d / A_ef
print('A_ef:', A_ef)
print('sigma_c90_appoggio:', sigma_c90_appoggio)

A_ef: 22000
sigma_c90_appoggio: 6.140969318181819


In [22]:
print('R:', k_c90*f_c90d)
sigma_c90_appoggio < k_c90 * f_c90d

R: 2.7155172413793105


False

In [23]:
f_c90d

1.5517241379310345

## SLE 

In [15]:
q = [0.93*4.33+0.96+1.65*4.33+0.8*4.33,2*4.33]

chi = 1.2
def w_inst(q):
    #chi = 1.2
    
    w_inst_M = 5/384 * q * l**4 / (E0mean * J)
    w_inst_V = chi * 1/8 * q * l**2 / (Gmean * b * h)
    w_inst = w_inst_M + w_inst_V
    
    return w_inst

w=[]
for qq in q:
    w_i = w_inst(qq)
    w.append(w_i)
    print(w_i)
    print(l/w_i)
    print('\n')

print('w_inst totale = ',np.sum(w))
print(l/250)
print(l/500)

11.07836208141602
689.1813017023276


6.151725228276462
1241.1152508739585


w_inst totale =  17.230087309692482
30.54
15.27


In [20]:
q

[15.5954, 8.66]

In [16]:
w_inst(1)

0.7103608808633328

In [17]:
w

[11.07836208141602, 6.151725228276462]

### Da sistemare. la formula non è cosi per il secondo carico variabile!

In [18]:
k_def = 0.6
psi2 = ['g1-g2', 0.3] #psi dei carichi variabili 

w_netFin = w[0]*(1 + k_def) 
print(w_netFin)
for i in range(1,len(w)):
    w_netFin = w_netFin + w[i]*(1 + psi2[i]*k_def)
    print(w_netFin)
w_netFin

17.725379330265636
24.98441509963186


24.98441509963186

In [19]:
l/500

15.27